Кобзарь О.С. Хабибуллин Р.А. 2019 г.

# Модель фонтанирующей скважины

In [ ]:
import sys
sys.path.append('../')

import uniflocpy.uWell.deviation_survey as dev_sev
import uniflocpy.uTools.data_workflow as utool
import uniflocpy.uWell.uPipe as Pipe
import uniflocpy.uWell.Self_flow_well as self_flow_well
import plotly.graph_objs as go
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
import pandas as pd
init_notebook_mode(connected=True)
import scipy.interpolate as interpolate
import matplotlib.pyplot as plt


In [ ]:
simple_well = self_flow_well.self_flow_well(h_intake_mes_m=999,h_intake_vert_m=999,
                                            h_bottomhole_mes_m=1000,h_bottomhole_vert_m=1000,
                                            t_earth_init_in_reservoir_c=90, t_bottomhole_c=92,
                                            p_bottomhole_bar=30)
simple_well.well_work_time_sec = 1 

In [ ]:
simple_well.calc_all_from_down_to_up()

In [ ]:
result_df = simple_well.data.combine_df_from_objects(simple_well.data.df_list)

In [ ]:
result_df.to_excel('test.xlsx')

In [ ]:
for i in result_df.columns:
    print(i)

In [ ]:
result_df.index = result_df['well.h_calculated_mes_m']
patterns = ["d_m"]

In [ ]:
def find_by_patterns(patterns, list_to_search):
    res = [x for x in list_to_search if re.search(patterns[0], x)] 
    if len(patterns) >1:
        for i in patterns[1:]:
            res = [x for x in res if re.search(i, x)] 
    return res

In [ ]:
res = find_by_patterns(patterns, list(result_df.columns))
res

In [ ]:
re.findall("mu", result_df.columns[0])

In [ ]:
result_df[res].plot()

In [ ]:
result_df['well.p_calculated_bar'].T.plot()

In [ ]:
def extract_data(this_class, init_class_name):
    all_class_dicts_well = []
    def extract_recursicely_data_from_obj(this_class):
        this_class_dict = this_class.__dict__
        for i, j in this_class_dict.items():
            if "uniflocpy" in str(type(j)):
                this_class_dict = extract_recursicely_data_from_obj(j)
                all_class_dicts_well.append({i:this_class_dict})
        return this_class_dict
    ___ = extract_recursicely_data_from_obj(this_class)
    all_class_dicts_well.append({init_class_name: this_class.__dict__})
    return all_class_dicts_well


def combine_object_dicts_into_one(data):
    super_dict = {}
    casing_pipe_object_not_passed = True
    for i in data:
        if list(i.keys())[0] != 'data':
            object_name = list(i.keys())[0]
            if object_name == "casing_pipe":
                casing_pipe_object_not_passed = False
            if object_name != "well_profile" and object_name != "well" and casing_pipe_object_not_passed:
                parameter_mark = f"casing_pipe.{object_name}."
            elif object_name != "well_profile" and object_name != "well" and not casing_pipe_object_not_passed:
                parameter_mark = f"tube_pipe.{object_name}."
            else:
                parameter_mark = f"{object_name}."
            for j,k in i[object_name].items():
                super_dict[parameter_mark+j] = k
    return super_dict

def leave_numbers(data):
    new_data = {}
    for i, j in data.items():
        if str(type(j)) == "<class 'numpy.ndarray'>":
            
            if len(j) == 1:
                new_data[i] = float(j)
            else:
                pass
        elif type(j) == int or type(j) == float or str(type(j)) == "<class 'numpy.float64'>" or str(type(j)) == "<class 'NoneType'>" or type(j) == bool:
            new_data[i] = j
        else:
            pass
    return new_data

def extract_data_from_object(this_object, object_name, index_to_df = [0]):
    data = extract_data(simple_well, object_name)
    data = combine_object_dicts_into_one(data)
    data = leave_numbers(data)
    df = pd.DataFrame(data, index = index_to_df)
    return df

def combine_df_from_objects(df_list):
    for i in df_list:
        try:
            result_df = result_df.append(i)
        except:
            result_df = df_list[0]
    return result_df

In [ ]:
df = extract_data_from_object(simple_well, 'well', [0])

In [ ]:
df

In [ ]:
for i in df.columns:
    print(i)

In [ ]:
simple_well.data.print_all_names()

In [ ]:
def trace(data_x, data_y, namexy):
    tracep = go.Scattergl(
        x = data_x,
        y = data_y,
        name = namexy,
        mode = 'lines'
    )
    return tracep

def plot():
    layout = dict(title = 'Расчет фонтанирующей скважины',  yaxis=dict(title = 'Измеренная глубина, м', autorange='reversed'))

    fig = dict(data=data, layout=layout)

    iplot(fig, filename='basic-scatter')

In [ ]:
wells = []
dfs = []
for i in range(1, 300, 50):
    simple_well = self_flow_well.self_flow_well(h_intake_mes_m=999,h_intake_vert_m=999,
                                            h_bottomhole_mes_m=1000,h_bottomhole_vert_m=1000,
                                            t_earth_init_in_reservoir_c=90, t_bottomhole_c=92,
                                            d_casing_inner_m=0.12,
                                            d_tube_inner_m=0.062,
                                            qliq_on_surface_m3day = 50,
                                                p_bottomhole_bar=50
                                       )
    simple_well.casing_pipe.fluid_flow.rsb_m3m3 = 10 * i
    simple_well.tube_pipe.fluid_flow.rsb_m3m3 = 10 * i
    simple_well.casing_pipe.fluid_flow.fl.rsb_m3m3 = 10 * i
    simple_well.tube_pipe.fluid_flow.fl.rsb_m3m3 = 10 * i
    simple_well.well_work_time_sec = 1 
    simple_well.calc_all_from_down_to_up()
    df = extract_data_from_object(simple_well, 'well', [i])
    dfs.append(df)
    wells.append(simple_well)

In [ ]:
result_df =combine_df_from_objects(dfs)

In [ ]:
result_df

In [ ]:
traces = []
for simple_well in wells:
    trace1 = trace(simple_well.data.get_values(25), simple_well.data.get_values(24),  "Давление, бар")
    traces.append(trace1)
data=traces
plot()    

In [ ]:
trace1 = trace(simple_well.data.get_values(28), simple_well.data.get_values(24),  "Температурный градиент, С/м")
trace2 = trace(simple_well.data.get_values(29), simple_well.data.get_values(24),  "Градиент давления, бар/м")
data=[trace1,trace2]
plot()

In [ ]:
trace1 = trace(simple_well.data.get_values(25), simple_well.data.get_values(24),  "Давление, бар")
trace2 = trace(simple_well.data.get_values(26), simple_well.data.get_values(24),  "Температура флюида, С")
trace3 = trace(simple_well.data.get_values(27), simple_well.data.get_values(24),  "Температура земли, С")
data=[trace1,trace2,trace3]
plot()

In [ ]:
trace1 = go.Scatter3d(
    x=simple_well.well_profile.x_displacement_m,
    y=simple_well.well_profile.x_displacement_m * 0,
    z=simple_well.well_profile.h_vert_m * (-1),  # перевод в альтитуду,
    marker=dict(
        size=4,
        colorscale='Viridis',
    ),
    name = 'Построенный профиль',
    )

trace2 = go.Scatter3d(
    x=simple_well.well_profile.get_x_displacement_m(simple_well.well_profile.h_mes_init_data_for_interpolation_m),
    y=simple_well.well_profile.get_x_displacement_m(simple_well.well_profile.h_mes_init_data_for_interpolation_m) * 0,
    z=simple_well.well_profile.h_vert_init_data_for_interpolation_m * (-1),
    mode = 'markers',
    marker = dict(
        size=10
    ),
    name = 'Исходные точки',
    )

data = [trace1, trace2]


layout = dict(title = '3D профиль простой скважины',  yaxis=dict(autorange='reversed'))
fig = dict(data=data, layout=layout)
iplot(fig, filename='simple-3d-scatter')